# Проектирование экспериментов

## Введение в проектирование экспериментов

In [25]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms

import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

#Загрузка данных
df = pd.read_csv('data/KaggleV2-May-2016.csv')
display(df.head())


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [26]:
#Определим соотношение людей которые пропускают приемы,
#к тем, кто этого не делает с помощью гистограммы

fig = px.histogram(
    data_frame=df,
    x='No-show',
    title= 'Соотношение людей, которые пропускают приемы, к тем, кто этого не делает',
    width=1000,
    height=600
)
fig.show()

DEBUG: 2025-07-27 16:30:46,457: https://www.comet.com:443 "GET /clientlib/health/ping HTTP/1.1" 200 67


In [27]:
#Или можно получить точную пропорцию при помощи метода value_counts()
df['No-show'].value_counts(True)

No-show
No     0.798067
Yes    0.201933
Name: proportion, dtype: float64

In [28]:
#При помощи этого метода определим, кто чаще не является на прием - мужчины или женщины
df[df['No-show'] == 'Yes']['Gender'].value_counts(True)

Gender
F    0.653882
M    0.346118
Name: proportion, dtype: float64

Итак мы видим, что людей, пропускающих прием, примерно 20%. Из них около 65% женщины.

In [29]:
#Для обучения модели машинного обучения, необходимо закодировать столбцы
#'Gender' и 'Neighbourhood', так как для ММО необходимы числовые признаки.
categorical_columns_names = ['Gender', 'Neighbourhood']
encoded_columns = pd.get_dummies(df, columns= categorical_columns_names)

## Логирование экспериментов

In [30]:
#Импортирование модуля logging для создания логирования
import logging

### Настройка уровня логирования

In [31]:
#Установка уровня логирования
import logging
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

#Тут теперь отображаются тоже все 5 уровней, до этого было 3
#начиная с WARNING и выше

DEBUG: 2025-07-27 16:30:46,669: This is a debug message
INFO: 2025-07-27 16:30:46,671: This is an informational message
ERROR: 2025-07-27 16:30:46,674: This is an error message
CRITICAL: 2025-07-27 16:30:46,675: This is a critical message


In [32]:
#Перезапустим ноутбук чтобы сбросить настройки logging
      #перезапустили
#Изменим конфигурацию логгера, чтобы вывести все урвони
import logging
logging.basicConfig(level=logging.DEBUG)

#Установим уровни логирования
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

DEBUG: 2025-07-27 16:30:46,684: This is a debug message
INFO: 2025-07-27 16:30:46,685: This is an informational message
ERROR: 2025-07-27 16:30:46,687: This is an error message
CRITICAL: 2025-07-27 16:30:46,687: This is a critical message


In [33]:
#Другой вариант сбросить настройки logging
#воспользоваться функцией reload()
from importlib import reload
import logging
reload(logging)

#Изменим конфигурацию логгера, чтобы вывести все урвони
logging.basicConfig(level=logging.DEBUG)

#Установим уровни логирования
logging.debug('This is a debug message')
logging.info('This is an informational message')
logging.warning('This message is a warning')
logging.error('This is an error message')
logging.critical('This is a critical message')

DEBUG:root:This is a debug message
INFO:root:This is an informational message
ERROR:root:This is an error message
CRITICAL:root:This is a critical message


### Установка обработчика (хендлера) логирования

In [34]:
#Воспользуемся обработчиком FileHandler, который возьмет запись журнала
#и добавит его в файл журнала
logging.FileHandler('log_file.log')

<FileHandler c:\Program Files (x86)\IDE\Skillfactory\EDA_5_Проектирование экспериментов\log_file.log (NOTSET)>

### Установка формата логирования

In [35]:
#Так как мы переопределяем настройки библиотеки logging,
#необходимо сбросить конфигурацию
from importlib import reload
import logging
reload(logging)
#Установим свой формат логирования
logging.basicConfig(
    format="%(levelname)s: %(asctime)s: %(message)s",
    level=logging.DEBUG
)
logging.info('Check')

INFO: 2025-07-27 16:30:46,742: Check


## Знакомство с Comet.ml

In [ ]:
#Пробуем создать первый эксперимент

from comet_ml import Experiment

# Создайте эксперимент с помощью вашего API ключа
experiment = Experiment(
  api_key="y3uZdZp0cZQ4bpFjgpCplMGaI",
  project_name="medical-appointment",
  workspace="ret4ed"
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : particular_grove_3021
COMET INFO:     url                   : https://www.comet.com/ret4ed/medical-appointment/f85eb055ff7c4fec8599e89d56c2a191
COMET INFO:   Uploads:
COMET INFO:     environment details : 1


COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
DEBUG: 2025-07-27 16:30:46,916: Resetting dropped connection: www.comet.com
DEBUG: 2025-07-27 16:30:46,922: https://www.comet.com:443 "POST /clientlib/notification/experiment HTTP/1.1" 200 74
DEBUG: 2025-07-27 16:30:46,925: Starting new HTTPS connection (2): www.comet.com:443
DEBUG: 2025-07-27 16:30:47,352: https://www.comet.com:443 "GET /api/rest/v2/experiment/metadata?experimentKey=f85eb055ff7c4fec8599e89d56c2a191 HTTP/1.1" 200 325
DEBUG: 2025-07-27 16:30:47,355: https://www.comet.com:443 "POST /clientlib/rest/v2/write/experiment/output HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:47,484: Starting new HTTPS connection (2): www.comet.com:443
DEBUG: 2025-07-27 16:30:47,505: https://www.comet.com:443 "POST /clientlib/status-report/update HTTP/1.1" 200 159
COMET WARNING: As you are 

DEBUG: 2025-07-27 16:30:49,750: https://www.comet.com:443 "POST /api/rest/v2/write/experiment/file-path HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:49,951: https://www.comet.com:443 "POST /clientlib/rest/v2/write/experiment/system-details HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:50,118: https://www.comet.com:443 "POST /clientlib/rest/v2/write/experiment/system-details HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:50,302: https://www.comet.com:443 "POST /clientlib/batch/logger/experiment/parameter HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:50,467: https://www.comet.com:443 "POST /api/rest/v2/write/experiment/git/metadata HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:50,652: https://www.comet.com:443 "POST /clientlib/batch/logger/experiment/metric HTTP/1.1" 200 55
DEBUG: 2025-07-27 16:30:51,102: https://www.comet.com:443 "GET /clientlib/health/ping HTTP/1.1" 200 67
DEBUG: 2025-07-27 16:30:53,135: https://www.comet.com:443 "GET /clientlib/health/ping HTTP/1.1" 200 67
DEBUG: 2025-07-27 16:30:55,149: http

In [39]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : creative_salamander_8540
COMET INFO:     url                   : https://www.comet.com/ret4ed/medical-appointment/4e9b74bf1e4c468fbe81794442025845
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     git metadata        : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
DEBUG: 2025-07-27 16:45:48,520: https://www.comet.com:443 "POST /clientlib/notification/experiment HTTP/1.1" 200 74
DEBUG: 2025-07-27 16:45:48,534: Resetting dropped connection: www.comet.com
DEBUG: 2025-07-27 16:45:49,011: https://www